In [22]:
import pandas as pd
from tipo_stats import TipoStats

def get_target(max_value_item, valor_item, tipo_stats):
    '''
    Função para calcular o valor que sera usado para determinar quanto de stats sera necessario para chegar no maximo valor do item
    ex: max_value por ser lv 150 e spd é 15, valor_item é 10, entao eu preciso de 9(target). 9/2 = 4. O que somado com o valor do item 10 + 4 = 14. 1 a menos do que o max_value
    '''
    if tipo_stats in [TipoStats.MDR, TipoStats.PDR, TipoStats.CritChance, 
                    TipoStats.AddHit, TipoStats.AddShot, TipoStats.Parry, 
                    TipoStats.MagicPower]:
        target = ((max_value_item - valor_item) * 2) - 0.01
    else:
        target = ((max_value_item - valor_item) * 2) - 1
    
    print("Valor item:", valor_item)
    print("Target:", target)

    return target

# Função para ler os valores de um arquivo txt
def ler_valores(nome_arquivo):
    with open(nome_arquivo, 'r') as file:
        valores = [float(line.strip()) for line in file.readlines()]
    return valores

def get_max_value_item(item_lv, tipo_stats: TipoStats):
    '''
    Obtem o valor maximo do item de acordo com seu tipo de status e item lv. 
    Ex: item com do tipo SPD e lv 150, seu max_value é 15
    '''
    df = pd.read_csv("docs/stats.csv")
    max_value_item = df[df["Item Lv"] == item_lv][tipo_stats.value].iloc[0]
    try:
        # Verifica se o tipo_stats está na lista de tipos que devem ser float
        if tipo_stats in [TipoStats.MDR, TipoStats.PDR, TipoStats.CritChance, 
                          TipoStats.AddHit, TipoStats.AddShot, TipoStats.Parry, 
                          TipoStats.MagicPower]:
            max_value_item = float(max_value_item)  # Converte para float
        else:
            max_value_item = int(max_value_item)  # Converte para int
    except (ValueError, TypeError):
        # Se a conversão falhar, define um valor padrão ou levanta uma exceção
        print(f"Erro: Valor inválido para {tipo_stats}: {max_value_item}")
        max_value_item = 0  # Ou levante uma exceção, se preferir
       
    print("Max value item:", max_value_item)

    return max_value_item

from itertools import combinations

def encontrar_melhor_combinacao(valores, target, tipo_stats : TipoStats):
    '''
    Qual é a melhor combinação que somada seja o mais proximo possivel do target, segundo a lista de valores passada?
    Caso a todas as somas possiveis ultrapassem o target, trazer o valor mais proximo sem soma-los.
    Valor nao pode exceder o target
    '''
    # Melhor combinação encontrada
    melhor_soma = 0
    melhor_combinacao = []

    if tipo_stats == TipoStats.BonusDmg:
        valores = [valor-10 for valor in valores]

    # Testa todas as combinações possíveis
    for r in range(1, len(valores) + 1):
        for combinacao in combinations(valores, r):
            soma_atual = sum(combinacao)
            if soma_atual <= target and soma_atual > melhor_soma:
                melhor_soma = soma_atual
                melhor_combinacao = combinacao
    
    if tipo_stats == TipoStats.BonusDmg:
        melhor_combinacao = [comb+10 for comb in melhor_combinacao]

    # Exibir resultado
    print("Melhor combinação:", melhor_combinacao)
    print(f"Soma mais próxima de {target}:", melhor_soma)
    print("Diferença:", target - melhor_soma)
    
# Função para encontrar a melhor combinação
def encontrar_melhor_combinacao_valores_subsequentes(valores, target, tipo_stats):
    '''
    Tem algum dos valores da lista que somando ele + (outro valor / 2) da o target ou um valor aproximado? A soma não pode exceder o target.
    '''
    melhor_combinacao = None
    melhor_diferenca = float('inf')

    if tipo_stats == TipoStats.BonusDmg:
        valores = [valor-10 for valor in valores]

    for v1 in valores:
        for v2 in valores:
            if v1 != v2:  # Evitar usar o mesmo valor para dividir por 2
                soma = v1 + (v2 / 2)
                if soma <= target:
                    diferenca = target - soma
                    if diferenca < melhor_diferenca:
                        melhor_diferenca = diferenca
                        melhor_combinacao = (v1, v2, soma)

    if melhor_combinacao and tipo_stats == TipoStats.BonusDmg:
        melhor_combinacao = [comb+10 for comb in melhor_combinacao]
        
    print(f'Melhor combinação: {melhor_combinacao}')
    
    return melhor_combinacao

In [23]:
item_lv = 200
tipo_stats = TipoStats.QItemFind
valor_item = 56

max_value_item = get_max_value_item(item_lv, tipo_stats)

target = get_target(max_value_item, valor_item, tipo_stats)

# Ler valores do arquivo
filename = f'docs/{tipo_stats.value}.txt'  # Ajuste o caminho conforme necessário
lista_valores_arquivo = ler_valores(filename)
#Remover valor_item da lista, já que será usado um item da propria lista
if valor_item in lista_valores_arquivo:
    lista_valores_arquivo.remove(valor_item)

Max value item: 170
Valor item: 56
Target: 227


In [24]:
melhor_combinacao = encontrar_melhor_combinacao(lista_valores_arquivo, target, tipo_stats)

Melhor combinação: (48.0, 44.0, 86.0, 43.0)
Soma mais próxima de 227: 221.0
Diferença: 6.0


In [25]:
melhor_combinacao = encontrar_melhor_combinacao_valores_subsequentes(lista_valores_arquivo, target, tipo_stats)

Melhor combinação: (152.0, 114.0, 209.0)
